# Arquitetura e Treinamento de uma MLP (visão guiada)

**Objetivo da aula (30–45 min):**
- Entender a arquitetura básica de uma MLP para classificação binária.
- Compreender o *forward*, *loss* (BCE), *backprop* e um laço de treino em **NumPy**.
- Realizar micro-exercícios para fixar conceitos e leitura de *shapes*.

**Plano rápido**
1. Camada densa e não linearidade.
2. Ativações (`ReLU`, `Sigmoid`) e gradientes.
3. *Forward* vetorizado (2 camadas).
4. Função de custo BCE + métrica de acurácia.
5. *Backpropagation* vetorizada.
6. *Training loop* curto e observação da curva de perda.

## 1) Camada densa mínima (conceito)

A camada densa (totalmente conectada) aplica \(Z = XW + b\).  
Se **X** tem forma `(m, d)` e a camada possui `h` neurônios, então **W** tem forma `(d, h)` e **b** tem forma `(1, h)`.

**Micro-exercício 1 (responda em texto em uma célula abaixo):**
- Se a entrada tem `d=6` atributos e a camada oculta tem `h=10` neurônios, qual a forma de `W` e `b`?
- Explique por que a presença de uma **não linearidade** entre camadas é necessária para modelar funções não lineares.

## 2) Funções de ativação e derivadas

Vamos implementar `ReLU` e `Sigmoid`, além de seus gradientes. Observe que o gradiente da Sigmoid pode ser expresso em função da **saída ativada**:

Seja 
$$a = \sigma(z) = \frac{1}{1 + e^{-z}}$$

então a derivada é:
$$\frac{d\,\sigma(z)}{dz} = \sigma(z)\,(1-\sigma(z)) = a(1-a)$$

In [ ]:
import numpy as np

def relu(z):
    return np.maximum(0, z)

def relu_grad(z):
    return (z > 0).astype(float)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_grad(a):
    # a = sigmoid(z)
    return a * (1.0 - a)

**Micro-exercício 2:**  
Por que é prático implementar `sigmoid_grad` recebendo a **ativação `a`** em vez de `z`?

## 3) Forward propagation (duas camadas)

Consideraremos uma MLP com **1 camada oculta** (ReLU) e **camada de saída** (Sigmoid) para classificação binária.

In [ ]:
def forward(X, params):
    W1, b1 = params["W1"], params["b1"]
    W2, b2 = params["W2"], params["b2"]
    Z1 = X @ W1 + b1
    A1 = relu(Z1)
    Z2 = A1 @ W2 + b2
    A2 = sigmoid(Z2)  # probabilidade classe=1
    cache = {"X": X, "Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

**Micro-exercício 3:**  
Onde acontece a **não linearidade** no `forward` e qual o seu papel?

## 4) Função de custo (BCE) e métrica de acurácia

Usaremos a **Binary Cross-Entropy** (BCE) estável numericamente. Para evitar `log(0)`, aplicamos *clipping* nas probabilidades.

In [ ]:
def bce_loss(y_true, y_pred, eps=1e-12):
    y_pred = np.clip(y_pred, eps, 1.0 - eps)
    return -np.mean(y_true*np.log(y_pred) + (1.0 - y_true)*np.log(1.0 - y_pred))

def accuracy(y_true, y_pred, thr=0.5):
    return np.mean((y_pred >= thr) == y_true)

**Micro-exercício 4:**  
Explique o papel do `np.clip` na `bce_loss`. O que pode acontecer sem esse cuidado?

## 5) Backpropagation (derivadas em cadeia)

Abaixo implementamos os gradientes vetorizados para uma MLP de duas camadas (BCE + Sigmoid na saída).  
Observação: dividimos por `m` (tamanho do lote) para obter o gradiente **médio**.

In [ ]:
def backward(y_true, cache, params):
    W2 = params["W2"]
    X, Z1, A1, Z2, A2 = cache["X"], cache["Z1"], cache["A1"], cache["Z2"], cache["A2"]
    m = y_true.shape[0]

    # dL/dZ2 para BCE+Sigmoid com y_pred = A2
    dZ2 = (A2 - y_true) / m
    dW2 = A1.T @ dZ2
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = dZ2 @ W2.T
    dZ1 = dA1 * relu_grad(Z1)
    dW1 = X.T @ dZ1
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    return {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

**Micro-exercício 5:**  
Por que usamos a **regra da cadeia** para obter `dZ1` a partir de `dA1`? Explique o papel de `relu_grad(Z1)`.

## 6) Laço de treinamento curto + curvas

Vamos inicializar os parâmetros, treinar por poucas épocas e observar a curva de perda.

In [ ]:
def init_params(n_in, n_hidden, n_out, seed=42):
    rng = np.random.default_rng(seed)
    # inicialização com variância ~ 1/fan_in
    W1 = rng.normal(0, 1.0/np.sqrt(n_in), size=(n_in, n_hidden))
    b1 = np.zeros((1, n_hidden))
    W2 = rng.normal(0, 1.0/np.sqrt(n_hidden), size=(n_hidden, n_out))
    b2 = np.zeros((1, n_out))
    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}

def sgd_step(params, grads, lr=0.1):
    for key in ["W1","b1","W2","b2"]:
        params[key] -= lr * grads["d"+key]

def train(X, y, n_hidden=8, lr=0.1, epochs=100, seed=42, verbose=False):
    params = init_params(X.shape[1], n_hidden, 1, seed=seed)
    losses = []
    for ep in range(epochs):
        y_pred, cache = forward(X, params)
        loss = bce_loss(y, y_pred)
        grads = backward(y, cache, params)
        sgd_step(params, grads, lr)
        losses.append(loss)
        if verbose and (ep % max(1, epochs//10) == 0):
            print(f"época {ep:4d} | loss={loss:.6f}")
    return params, np.array(losses)

**Dados sintéticos para demonstração**  
(Aqui criamos um conjunto simples apenas para visualizar o comportamento do treinamento.)

In [ ]:
# Dados sintéticos binários (duas nuvens separáveis com ruído leve)
rng = np.random.default_rng(0)
m = 400
X_pos = rng.normal([2.0, 2.0], [1.0, 1.0], size=(m//2, 2))
X_neg = rng.normal([-2.0, -2.0], [1.0, 1.0], size=(m//2, 2))
X = np.vstack([X_pos, X_neg])
y = np.vstack([np.ones((m//2,1)), np.zeros((m//2,1))])

# Embaralhar
idx = rng.permutation(m)
X, y = X[idx], y[idx]

In [ ]:
# Treinar com diferentes taxas de aprendizado
import matplotlib.pyplot as plt

for lr in [0.01, 0.05, 0.1]:
    _, losses = train(X, y, n_hidden=8, lr=lr, epochs=200, seed=0, verbose=False)
    plt.figure()
    plt.plot(losses)
    plt.xlabel("Época")
    plt.ylabel("BCE")
    plt.title(f"Curva de treino (lr={lr})")
    plt.show()

**Micro-exercício 6:**  
Compare o comportamento da curva de perda para `lr = 0.01`, `0.05` e `0.1`.  
- Qual apresenta convergência mais lenta?  
- Em algum caso a perda oscila? O que isso sugere sobre a escolha de `lr`?

## 7) Checklist mental (projeto de RNAs)

- **Entradas/saídas**: tipos, escalas e codificação dos rótulos.
- **Arquitetura**: número de camadas e neurônios, ativação.
- **Objetivo**: função de perda e métricas.
- **Treinamento**: inicialização, `lr`, épocas, regularização.
- **Avaliação**: acurácia, matriz de confusão, inspeção de erros.
- **Iteração**: experimente `n_hidden`, `lr` e inicializações diferentes.